In [178]:
import pandas as pd
from pandas import Series
import numpy as np
from joblib import Parallel, delayed
import json
from scipy.sparse import hstack
import time
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from sklearn.preprocessing.data import OneHotEncoder
from sklearn import preprocessing

In [31]:
now_year

'2020-01-02 15:53:11'

In [123]:
#年龄处理
time_now = int(time.time())
time_local = time.localtime(time_now)
now_year = time.strftime("%Y-%m-%d %H:%M:%S",time_local)
now_year = int(now_year[:4])
def age_process(birthday):
    #len(list(user_df.loc[2,'birthday'])) = 12,后三位为毫秒
    try:
        time_local = time.localtime(int(birthday[:9]))
    except:
        return 4
    birth_year = time.strftime("%Y-%m-%d %H:%M:%S",time_local)
    age = now_year - int(birth_year[:4])
    if age==0:
        return 0
    elif age<18:
        return 1
    elif age<20:
        return 3
    elif age<22:
        return 4
    elif age<24:
        return 5
    elif age<26:
        return 6
    elif age<28:
        return 7
    elif age<30:
        return 8
    elif age<32:
        return 9
    elif age<34:
        return 10
    elif age<36:
        return 11
    elif age<38:
        return 12
    elif age<45:
        return 13
    elif age<60:
        return 14
    else:
        return 15

def register_type_fun(row):
    register_type_list = row.split(',')
    tmp_list = [0,0,0,0,0,0]
    if 'weixin' in register_type_list:
        tmp_list[0] = 1
    if 'sns' in register_type_list:
        tmp_list[1] = 1
    if 'sina_weibo' in register_type_list:
        tmp_list[2] = 1
    if 'qq_zone' in register_type_list:
        tmp_list[3] = 1
    if 'missing' in register_type_list:
        tmp_list[5] = 1
    if 'cellphone' in register_type_list:
        tmp_list[4] = 1
    if 'xiaomi' in register_type_list:
        tmp_list[5] = 1
    if 'device' in register_type_list:
        tmp_list[5] = 1
    return tmp_list

def columbus_user_tags_fun(row):
    columbus_user_tags_list = row.split(',')
    tmp_list = [0,0,0,0,0,0]
    if 'weixin' in register_type_list:
        tmp_list[0] = 1
    if 'sns' in register_type_list:
        tmp_list[1] = 1
    if 'sina_weibo' in register_type_list:
        tmp_list[2] = 1
    if 'qq_zone' in register_type_list:
        tmp_list[3] = 1
    if 'missing' in register_type_list:
        tmp_list[5] = 1
    if 'cellphone' in register_type_list:
        tmp_list[4] = 1
    if 'xiaomi' in register_type_list:
        tmp_list[5] = 1
    if 'device' in register_type_list:
        tmp_list[5] = 1
    return tmp_list

def time_process(data_time):
    try:
        time_tmp = time.localtime(int(data_time[:9]))
    except:
        return 4
    time_tmp = time.strftime("%Y-%m-%d %H:%M:%S",time_tmp)
    year = list(time_tmp)[:4]
    month = list(time_tmp)[5:7]
    day = list(time_tmp)[9:11]
    hour = list(time_tmp)[3]
    return year,month,day,hour 

def activate_channel_label_fun(slug):
    slug = str(slug)
    if slug=='record_eating_activity':
        return 1
    elif slug=='record_weight':
        return 2
    elif slug=='post_create':
        return 3
    elif slug=='habit_checkin':
        return 4
    elif slug=='search_food':
        return 5
    elif slug=='finished_sports':
        return 6
    else:  
        return 0
    
def special_condition_label_fun(x):
    x = str(x)
    if x=='0.0' or x=='0':
        return 0
    elif x=='1.0' or x=='1':
        return 1
    elif x=='2.0' or x=='2':
        return 2
    elif x=='3.0' or x=='3':
        return 3
    else:  
        return 4

# 四分位箱线图处理异常值
def box_plot(data_series,feature_name):
    data_series = pd.Series(sorted(data_series))
    #q_abnormal_low = data_series.quantile(0.25) - 1.5 * (data_series.quantile(0.75) - data_series.quantile(0.25))
    q_abnormal_up = data_series.quantile(0.90) + 1.5 * (data_series.quantile(0.90) - data_series.quantile(0.25))
    q_abnormal_up = int(round(q_abnormal_up,0))
    data_series = data_series.apply(lambda x:q_abnormal_up if x>q_abnormal_up else x)
    return data_series
    
def _load_data(data_file):
    user_df = pd.read_csv(filepath_or_buffer = data_file,sep=",",names=user_feature_list)
    user_df.drop(labels=None,axis=0, index=0, columns=None, inplace=True)
    return user_df


In [10]:
data_file_list = [
'/Users/wjj/Desktop/jupyter/Boohee_GBDT+LR/data/0102/用户反例原始数据/view_comment_not_add_buy_user_12.csv',

]
user_feature_list = [
 'second_id','slug','time','birthday','register_timestamp','first_visit_time','survey_finished_at',
 'first_traffic_source_type','register_source_app_key','register_type',
 'register_sns','gender','phone_model','app_channel','app_version','os_version','province','city_level','purpose',
 'target_weight','start_weight','latest_weight','bmi','budget_calory',
 'view_goods_count','add_cart_count','commit_order_count','num_paid','eka_num_paid',
 'total_cost','recent_buy_at','record_food_count','search_food_count','record_weight_count',
 'finished_sports_count','habbit_checkin_count','post_create_count',
 'active_day_count','activate_channel','special_condition','easy_customer_first_buy_time',
 'easy_membership_first_platform','easy_membership_first_time','user_membership_status','user_fans_status',
 'guide_to_bh_at','wx_assistant_friend','columbus_user_tags'
]


In [159]:
user_df = pd.DataFrame(columns = user_feature_list)
for i in data_file_list:
    user_tmp_df =  _load_data(i)
    user_df = pd.concat([user_df,user_tmp_df],ignore_index=True)
user_df = user_df[~user_df['second_id'].isin(['second_id'])]
user_df = user_df.drop_duplicates('second_id')
del user_df['columbus_user_tags']

/Users/wjj/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (0,3,4,5,6,11,17,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,39,40,42,45) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


In [12]:
#phonemodel特征,不要重复跑，不然报错
#user_df['phone_model'] = user_df['phone_model'].fillna('1')
user_df.loc[(user_df["phone_model"]=='iPhone'),"phone_model"] = 2
user_df.loc[(user_df["phone_model"]!='iPhone'),"phone_model"] = 1

In [13]:
#purpose
user_df.loc[(user_df["purpose"]=='lost_weight'),"purpose_label"] = 1
user_df.loc[(user_df["purpose"]=='keep_muscle'),"purpose_label"] = 2
user_df.loc[(user_df["purpose"]=='gain_muscle'),"purpose_label"] = 3
user_df.loc[(user_df["purpose"]!='lost_weight') & (user_df["purpose"]!='keep_muscle') &(user_df["purpose"]!='gain_muscle'),"purpose_label"] = 4
user_df['purpose_label'] = user_df['purpose_label'].fillna(5)
del user_df['purpose']

In [14]:
#个人信息特征空值，不要重复跑，不然报错
user_df['city_level'].fillna(2,inplace=True)
user_df['city_level'] = user_df['city_level'].astype(np.int32)
#province_list = user_df['province'].value_counts().keys().tolist()[:37]
province_list=['广东省', '江苏省', '浙江省', '北京', '上海', '山东省', '河南省', '四川省', '湖北省', '河北省', '湖南省', '安徽省', '辽宁省', '福建省', '陕西省', '重庆', '江西省', '天津', '云南省', '黑龙江省', '山西省', '未知', '广西', '吉林省', '内蒙古', '贵州省', '新疆', '甘肃省', '海南省', '宁夏', '香港', '青海省', '台湾省', '澳门', '西藏']
user_df['province'].fillna('未知',inplace = True)
user_df['province'] = user_df['province'].apply(lambda x:x if x in province_list else '国外')
encoder = preprocessing.LabelEncoder()
encoder.fit(user_df['province'])
user_df['province'] = encoder.transform(user_df['province'])
user_df["gender"].fillna(2, inplace=True)
user_df['gender'] = user_df['gender'].apply(lambda x:1 if x=='1' or x==1 or x==1.0 else 2)
user_df["activate_channel"] = user_df["activate_channel"].apply(activate_channel_label_fun)
user_df["special_condition"] = user_df["special_condition"].apply(special_condition_label_fun)

In [15]:
#用户行为特征
feature_box_plot_list = ['view_goods_count', 'record_food_count', 'search_food_count','record_weight_count','active_day_count']                              
for i in feature_box_plot_list:
    user_df[i].fillna(0,inplace=True)
    user_df[i] = user_df[i].astype(np.int32)
    user_df[i] = box_plot(user_df[i],i)
    user_df[i].fillna(0,inplace=True)
    
# PND模型处理异常值
feature_custom_list = ['habbit_checkin_count','total_cost','num_paid','add_cart_count','eka_num_paid','commit_order_count','post_create_count','finished_sports_count']
for i in feature_custom_list:
    user_df[i].fillna(0,inplace=True)
    user_df[i] = user_df[i].astype(np.float32)
user_df['habbit_checkin_count'] = user_df['habbit_checkin_count'].apply(lambda x:200 if x>200 else x)
user_df['add_cart_count'] = user_df['add_cart_count'].apply(lambda x:80 if x>80 else x)
user_df['total_cost'] = user_df['total_cost'].apply(lambda x:5000 if x>5000 else x)
user_df['num_paid'] = user_df['num_paid'].apply(lambda x:20 if x>20 else x)
user_df['eka_num_paid'] = user_df['eka_num_paid'].apply(lambda x:20 if x>20 else x)
user_df['commit_order_count'] = user_df['commit_order_count'].apply(lambda x:30 if x>30 else x)
user_df['post_create_count'] = user_df['post_create_count'].apply(lambda x:50 if x>50 else x)
user_df['finished_sports_count'] = user_df['finished_sports_count'].apply(lambda x:80 if x>80 else x)
   

In [16]:
#用户体制特征使用平均填充
feature_body_list = ["target_weight","start_weight","latest_weight","bmi","budget_calory"]
for i in feature_body_list:
    user_df[i] = user_df[i].astype(np.float32)
    mean_val = user_df[i].mean()
    user_df[i].fillna(mean_val, inplace=True)
del mean_val

In [33]:
#时间特征
#年龄特征处理
user_df['birthday'] = user_df['birthday'].astype(str)
user_df['birthday'] = user_df['birthday'].apply(age_process)

In [144]:
user_df['register_sns'] = user_df['register_sns'].fillna('missing')
user_df= user_df.join(pd.get_dummies(user_df['register_sns'],prefix='register_sns_'))
del user_df['register_sns']

In [ ]:
#app_channel onehot
app_channel_list = ['missing', 'huawei','nearme', 'vivo','xiaomi','boohee','tencent','meizu', '_360','baidu','wandoujia','undefind','lenovo','zhihuan','oppo','other',]
user_df['app_channel'] = user_df['app_channel'].apply(lambda x:'vivo' if x == 'oppo' else x)
user_df['app_channel'] = user_df['app_channel'].apply(lambda x:x if x in app_channel_list else 'other')
user_df= user_df.join(pd.get_dummies(user_df['app_channel'],prefix='app_channel_'))
del user_df['app_channel']

In [145]:
#注册方式onehot
user_df['register_type'] = user_df['register_type'].fillna('missing')
user_df['register_type']= pd.DataFrame(user_df['register_type'].apply(register_type_fun))
user_df['register_type_weixin'] = user_df['register_type'].apply(lambda x:x[0])
user_df['register_type_sns'] = user_df['register_type'].apply(lambda x:x[1])
user_df['register_type_sina_weibo'] = user_df['register_type'].apply(lambda x:x[2])
user_df['register_type_qq_zone'] = user_df['register_type'].apply(lambda x:x[3])
user_df['register_type_cellphone'] = user_df['register_type'].apply(lambda x:x[4])
user_df['register_type_missing'] = user_df['register_type'].apply(lambda x:x[5])
del user_df['register_type']

In [153]:
user_df['first_traffic_source_type'].fillna('missing',inplace=True)
user_df['first_traffic_source_type'] = user_df['first_traffic_source_type'].apply(lambda x:'direct' if x == '直接流量')
user_df['first_traffic_source_type'] = user_df['first_traffic_source_type'].apply(lambda x:'indirect' if x == '间接流量')
user_df= user_df.join(pd.get_dummies(user_df['first_traffic_source_type'],prefix='first_traffic_source_type_'))
del user_df['first_traffic_source_type']

In [ ]:
user_df['register_source_app_key'] = user_df['register_source_app_key'].apply(lambda x:x if x == 'ifood' or x == 'boohee_weixin' or x == 'one' or x == 'food' or x == 'nice' else 'missing')            
user_df= user_df.join(pd.get_dummies(user_df['register_source_app_key'],prefix='register_source_app_key_'))
del user_df['register_source_app_key']

In [158]:
user_df['easy_membership_first_platform'].fillna('missing',inplace=True)
user_df= user_df.join(pd.get_dummies(user_df['easy_membership_first_platform'],prefix='easy_membership_first_platform_'))
del user_df['easy_membership_first_platform']

APP       40287
wechat     1272
Name: easy_membership_first_platform, dtype: int64

In [167]:
user_df['user_membership_status'].fillna('missing',inplace=True)
user_df['user_membership_status'] = user_df['user_membership_status'].apply(lambda x:'member_expire' if x == 'member_canceled' else x)
user_df['user_membership_status'] = user_df['user_membership_status'].apply(lambda x:'member_expire' if x == 'member_inactive' else x)
user_df['user_membership_status'] = user_df['user_membership_status'].apply(lambda x:'member_ongoing' if x == 'associate_member' else x)
user_df= user_df.join(pd.get_dummies(user_df['user_membership_status'],prefix='user_membership_status_'))
del user_df['user_membership_status']

In [169]:
user_df['user_fans_status'].fillna('missing',inplace=True)
user_df= user_df.join(pd.get_dummies(user_df['user_fans_status'],prefix='user_fans_status_'))
del user_df['user_fans_status']

In [171]:
user_df['wx_assistant_friend'].fillna('missing',inplace=True)
user_df['wx_assistant_friend'] = user_df['wx_assistant_friend'].apply(lambda x:'yes' if x == '是' else x)
user_df= user_df.join(pd.get_dummies(user_df['wx_assistant_friend'],prefix='wx_assistant_friend_'))
del user_df['wx_assistant_friend']

In [ ]:
['second_id','slug','time','register_timestamp','first_visit_time','survey_finished_at',
 'app_version','os_version',
 'recent_buy_at','easy_customer_first_buy_time',
 'easy_membership_first_time',
 'guide_to_bh_at'
]

user_df['columbus_user_tags'].fillna('missing',inplace=True)
columbus_user_tags_list =[]
for row in user_df.itertuples():
    tmp_list = getattr(row, 'columbus_user_tags').split(',')
    columbus_user_tags_list = columbus_user_tags_list+tmp_list
print(len(set(columbus_user_tags_list)))
columbus_user_tags_set = set(columbus_user_tags_list)

In [193]:
def convert_str_datetime(df): 
    
    df.insert(loc=2, column= timestamp , value=pd.to_datetime(df.transdate, format='%Y-%m-%d %H:%M:%S.%f')) 

In [194]:
user_df(user_df['birthday'].isnull() == False)

TypeError: 'DataFrame' object is not callable

In [190]:
user_df['birthday']

0          842457600000
1          882374400000
2          744480000000
3          923932800000
4          558457200000
5          968256000000
6          777139200000
7          915120000000
8          510768000000
9          784051200000
10         987782400000
11         849110400000
12         965836800000
13         855158400000
14         972662400000
15         649436400000
16         610729200000
17        1017072000000
18         946828800000
19         629395200000
20         390585600000
21         800380800000
22         895334400000
23         325008000000
24         870019200000
25        1041696000000
26         683391600000
27         741974400000
28         596131200000
29         763228800000
              ...      
999825      6.29741e+11
999826      9.63763e+11
999827        8.352e+11
999831      8.86349e+11
999840      8.62762e+11
999855       8.7503e+11
999864      6.56179e+11
999867      9.96336e+11
999869      9.47606e+11
999877       9.7223e+11
999879      1.02

In [69]:
def fun_1(row):
    year,month,day,hour = time_process(row)
    print(year,month,day,hour)
user_df['survey_finished_at'] = user_df['survey_finished_at'].astype(str)
user_df['survey_finished_at'].apply(fun_1)

['1', '9', '7', '4'] ['1', '2'] ['2', ' '] 4
['1', '9', '7', '4'] ['1', '2'] ['7', ' '] 4
['1', '9', '7', '4'] ['1', '2'] ['5', ' '] 4
['1', '9', '7', '4'] ['1', '2'] ['7', ' '] 4
['1', '9', '7', '4'] ['1', '2'] ['2', ' '] 4
['1', '9', '7', '4'] ['1', '2'] ['1', ' '] 4
['1', '9', '7', '4'] ['1', '2'] ['1', ' '] 4


TypeError: cannot unpack non-iterable int object

In [30]:
def time_process(data_time):
    try:
        time_tmp = time.localtime(int(data_time[:9]))
    except:
        return 4
    time_tmp = time.strftime("%Y-%m-%d %H:%M:%S",time_tmp)
    year = list(time_tmp)[:4]
    month = list(time_tmp)[5:7]
    day = list(time_tmp)[9:11]
    hour = list(time_tmp)[3]
    return year,month,day,hour 

'2020'